In [2]:
from gen_ai_hub.proxy.langchain import init_llm
import pandas as pd
from hana_ml import dataframe
from hana_ai.tools.toolkit import HANAMLToolkit


connection_context = dataframe.ConnectionContext(userkey="RaysKey")

llm = init_llm('gpt-4-32k', temperature=0.0, max_tokens=1800)
toolkit = HANAMLToolkit(connection_context, used_tools='all', return_direct={"fetch_data": False})

In [ ]:

toolkit.launch_mcp_server(server_name="test", sse_port=12345)

INFO:     Started server process [9064]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12346 (Press CTRL+C to quit)


In [4]:
from mcp.client.sse import sse_client  # SSE协议客户端
from mcp import ClientSession  # MCP会话管理

async def get_mcp_tools():
    """获取并打印MCP服务的所有工具列表"""
    try:
        # 1. 建立SSE连接
        async with sse_client("http://localhost:12346/sse") as streams:
            # 2. 创建会话
            async with ClientSession(*streams) as session:
                await session.initialize()  # 初始化会话
                
                # 3. 获取工具列表
                tools = await session.list_tools()
                print(f"✅ 成功获取 {len(tools.tools)} 个工具:")
                
                # 4. 打印工具详情
                for tool in tools.tools:
                    print(f"  🛠️ 名称: {tool.name}")
                    print(f"  📝 描述: {tool.description}")

                print("✅ 工具列表获取成功！")

                # 5. 调用fetch_data工具示例
                try:
                    # 调用工具
                    print("🔍 正在调用 fetch_data 工具...")
                    result = await session.call_tool(name="fetch_data", arguments={
                        "kwargs": {
                            "table_name": "PAL_COVID_DATA_TBL",  # 确保键名完全匹配
                            "top_n": 5
                        }
                    })
                except Exception as e:
                    # 捕获具体异常类型（如ValidationError）
                    print(f"❌ 工具调用失败: {type(e).__name__}: {str(e)}")
                return result

                
    except Exception as e:
        print(f"❌ 调用失败: {str(e)}")
        return None

# 执行异步函数
result = await get_mcp_tools()

INFO:     127.0.0.1:61639 - "GET /sse HTTP/1.1" 200 OK
INFO:     127.0.0.1:61641 - "POST /messages/?session_id=cdc7182578ee4178a90e13ce6009d60a HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:61641 - "POST /messages/?session_id=cdc7182578ee4178a90e13ce6009d60a HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:61641 - "POST /messages/?session_id=cdc7182578ee4178a90e13ce6009d60a HTTP/1.1" 202 Accepted
✅ 成功获取 16 个工具:
  🛠️ 名称: accuracy_measure
  📝 描述: To compute the accuracy measure using true and predict tables.
  🛠️ 名称: additive_model_forecast_fit_and_save
  📝 描述: To fit the additive model forecast and save the model in model storage.
  🛠️ 名称: additive_model_forecast_load_model_and_predict
  📝 描述: To load the additive model forecast from model storage and predict.
  🛠️ 名称: automatic_timeseries_fit_and_save
  📝 描述: To fit an AutomaticTimeseries model and save it in the model storage.
  🛠️ 名称: automatic_timeseries_load_model_and_predict
  📝 描述: To load a model and do the prediction using automatic tim

In [5]:
print(result.content[0].text)

     Date  Confirmed  Recovered  Deaths  Increase rate
1/22/2020        555         28      17            NaN
1/23/2020        654         30      18      17.837838
1/24/2020        941         36      26      43.883792
1/25/2020       1434         39      42      52.391073
1/26/2020       2118         52      56      47.698745
